In [ ]:
#splitting the text features, x_train_seq contains the processed text feature vector and y contains 
##  the respective emotions
x_train, x_test, y_train, y_test = train_test_split(x_train_seq_trunc,y,test_size=0.25,random_state=4)
x_train.shape

In [ ]:
##Splitting the speech features, M contains the processed speech feature vector and l contains 
##  the respective emotions
M_train, M_test, l_train, l_test = train_test_split(M,l,test_size=0.25,random_state=4)

In [ ]:
import keras
from keras.layers import Flatten,GRU,LSTM,Dense,Activation,Input,Dropout,Embedding,concatenate
from keras.utils import plot_model

In [ ]:
_input1   = Input(shape=(180,), dtype='float32')
_input12   = Dense(340,activation='relu')(_input1)
_input13   = Dropout(0.3)(_input12)
_input14   = Dense(470, activation='relu')(_input13)
_input15   = Dense(600, activation='relu')(_input14)
_input2   =  Input(shape=(MAX_LEN,))
_input21   = Embedding(NB_WORDS, GLOVE_DIM, input_length=MAX_LEN,weights=[emb_matrix],trainable=False)(_input2)
_input22   = LSTM(50,return_sequences=True)(_input21)
_input23   = LSTM(50,return_sequences=True)(_input22)
_input24   = Flatten()(_input23)
_input25   = Dropout(0.3)(_input24)

output1    = concatenate([_input15,_input25])
output2    = Dropout(0.3)(output1)
output3    = Dense(500, activation='relu')(output2)
output3    = Dense(4, activation='softmax')(output3)

model = keras.Model(inputs=[_input1,_input2], outputs=[output3])
model.summary()
plot_model(model, to_file='multiple_output.png')

In [ ]:
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
from keras.callbacks import ModelCheckpoint
filepath="weights.best.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')
callbacks_list = [checkpoint]
model.fit([M_train,x_train], y_train, epochs=150, batch_size=64,validation_data=([M_test,x_test], y_test),verbose=2,callbacks=callbacks_list)

In [ ]:
_, accuracy = model.evaluate([M_test,x_test], y_test)
print('Accuracy on test data: %.2f' % (accuracy*100))
_, accu = model.evaluate([M_train,x_train], y_train)
print('Accuracy on train data: %.2f' % (accu*100))

In [ ]:
import matplotlib.pyplot as plt
print(history.history.keys())
# summarize history for accuracy
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.legend(["Train", "Validation"])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.show()

# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.legend(["Train", "Validation"])
plt.title('model Loss')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.show()

In [ ]:
import keras
model.save("iemocap.h5")

In [ ]:
from sklearn import metrics         #confusion matrix
prediction=t.predict([M_test,x_test])
y_pred=(np.argmax(prediction, axis=1))
y_test=y_test.reshape((234,))
y_test=y_test.astype('int64') 
c = metrics.confusion_matrix(y_pred, y_test)
normed_c = (c.T / c.astype(np.float).sum(axis=1)).T

In [ ]:
import seaborn as sb
#fig= plt.figure(figsize=(11,11))
fig, (ax1) = plt.subplots(1,1,figsize=(6,4))
sb.set(font_scale=1)
x_axis_labels = ["Angry","Happy","Sad","Neutral"] # labels for x-axis
y_axis_labels = ["Angry","Happy","Sad","Neutral"]
#akws = {"ha": 'left',"va": 'top'}
cbar_kws = {"orientation":"vertical",  
           }
#plt.subplot(1,3,1)
sb.heatmap(normed_c,annot=True,cmap="YlGnBu",fmt='.3f',cbar_kws=cbar_kws,xticklabels=x_axis_labels,linewidths=1, yticklabels=y_axis_labels,ax=ax1)
#plt.subplot(1,3,2)
ax1.tick_params(rotation=0,labelsize=11.5)
plt.xlabel('Predicted Class')
plt.ylabel('True Class')
plt.savefig('confmat.pdf',bbox_inches ="tight", 
            pad_inches = 1, 
            orientation ='landscape',dpi=1200)
plt.show()